# Data Preprocessing

This notebook is for preprocessing. 
    -USA located English tweets gathered.
    -URLs, mentions, hashtags, special characters and stop words removed.
    -All text concerted to lowercase.
    -Lemmatization performed.

In [1]:
import pandas as pd
import numpy as np
import re
import glob
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
path = '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data'
all_files = glob.glob(path + "/*.csv")

In [5]:
all_files

['/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/20230101_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/20230219_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/1214_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/20230225_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/1225_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/0722_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/1008_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/UkraineCombinedTweetsDeduped_MAR30.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/0417_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/0504_UkraineCombinedTweetsDeduped.csv',
 '/home/erdenzehra/Deneme Tahtasi/Ukraine_Data/20230225_UkraineCombinedTweetsDeduped_processed.csv',
 '/home/erdenzeh

In [ ]:
all_files_split = np.array_split(all_files, 10)

for file_list in all_files_split:
    liste = []
    for filename in file_list:
        with open(filename, 'r', errors='ignore') as f:
            df = pd.read_csv(f, header=0, encoding='utf-8', on_bad_lines='skip', low_memory=False)
            liste.append(df)

In [7]:
def extract_usa_location(location):
    if pd.isna(location): 
        return ""
    for state in usa_states:
        if state in location.lower():
            return location
    return ""

def language_check(language):
    if pd.isna(language): 
        return False
    if language == 'en':
        return True
    else:
        return False
    
usa_states = [state.name.lower() for state in pycountry.subdivisions if state.country_code == 'US']

In [8]:
usa_states

['alaska',
 'alabama',
 'arkansas',
 'american samoa',
 'arizona',
 'california',
 'colorado',
 'connecticut',
 'district of columbia',
 'delaware',
 'florida',
 'georgia',
 'guam',
 'hawaii',
 'iowa',
 'idaho',
 'illinois',
 'indiana',
 'kansas',
 'kentucky',
 'louisiana',
 'massachusetts',
 'maryland',
 'maine',
 'michigan',
 'minnesota',
 'missouri',
 'northern mariana islands',
 'mississippi',
 'montana',
 'north carolina',
 'north dakota',
 'nebraska',
 'new hampshire',
 'new jersey',
 'new mexico',
 'nevada',
 'new york',
 'ohio',
 'oklahoma',
 'oregon',
 'pennsylvania',
 'puerto rico',
 'rhode island',
 'south carolina',
 'south dakota',
 'tennessee',
 'texas',
 'united states minor outlying islands',
 'utah',
 'virginia',
 'virgin islands, u.s.',
 'vermont',
 'washington',
 'wisconsin',
 'west virginia',
 'wyoming']

In [157]:
usa_location_list = []
for df in range(len(liste)):
    liste[df]["location"] = liste[df]["location"].fillna("")
    liste[df]["location"] = liste[df]["location"].astype(str)
    usa_locations_df = liste[df]["location"].apply(extract_usa_location).loc[lambda x: x != ""]
    usa_location_list.append(usa_locations_df)

In [158]:
usa_tweets_list = []
for i, usa_locations_df in enumerate(usa_location_list):
    usa_locations_indexes = usa_locations_df.index.tolist()
    usa_tweets_df = liste[i].loc[usa_locations_indexes]
    usa_tweets_list.append(usa_tweets_df)

In [159]:
bool_cols = usa_tweets_combined.select_dtypes(include=['object']).nunique() == 2
usa_tweets_combined.loc[:, bool_cols] = usa_tweets_combined.loc[:, bool_cols].astype(bool)

In [160]:
english_tweets = usa_tweets_combined[usa_tweets_combined['language'] == 'en']
english_tweets

,Unnamed: 0,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,...,original_tweet_id,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username
0,2409798,63352716,Cameron17z,Deaf Woman☕️ #IBelieveInGod🙏🏻 ☥ #DanceRap #Sup...,"Holyoke, Massachusetts USA🇺🇸",1073,561,87081,2009-08-06 03:14:19.000000,1500260228008448005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2409837,305661348,kristafrances,"I blame the GOP, Citizens United, and the Supr...","San Diego, California",3938,1505,713161,2011-05-26 15:54:47.000000,1500260243879731200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2409873,78400475,michikokakutani,"former chief book critic, The New York Times; ...","New York, NY",7266,113992,102855,2009-09-29 19:57:07.000000,1500260258274590728,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2409933,45673524,BrettaApplebaum,https://t.co/hF4XEIjH3U\n#Scientist #NastyWoma...,Alaska,8412,8537,625054,2009-06-08 21:17:27.000000,1500260283599638532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2409949,9449272,anakaoka,NaN,"Honolulu, Hawaii",245,962,23151,2007-10-15 05:22:58.000000,1500260289593241600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362800,190532,3302127160,AylaMeadow,"#write, #train others to speak, learn, #create...",New York,986,342,5088,2015-05-28 19:35:56,1577085773861769216,...,0,0,NaN,0,0,NaN,True,1577031631613087744,1498176452084256768,Podolyak_M
362801,190540,3302127160,AylaMeadow,"#write, #train others to speak, learn, #create...",New York,986,342,5088,2015-05-28 19:35:56,1577085864122818560,...,0,0,NaN,0,0,NaN,True,1577000192359821312,454099919,McFaul
362802,190546,2339297117,lu_nyc,NYCRunner/Political&CulturalJunkie/CDMX+NYC/Pr...,"New York, NY",892,128,3300,2014-02-12 13:22:20,1577085906342711298,...,0,0,NaN,0,16376937,errollouis,False,0,0,NaN
362803,190560,28018672,ImmaChocoholic,I've learned that people will forget what you ...,"Hartford, Connecticut",25636,27053,172717,2009-04-01 01:31:50,1577085980321849345,...,0,0,NaN,0,0,NaN,True,1577028615824252928,19706851,TelegraphWorld


In [161]:
URL_REGEX = re.compile(r'https?://\S+|www\.\S+')
MENTION_REGEX = re.compile(r'@\w+')
HASHTAG_REGEX = re.compile(r'#\w+')
SPECIAL_CHAR_REGEX = re.compile(r'[^\w\s]')

STOP_WORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

def clean_text(text):
    text = URL_REGEX.sub('', text)
    text = MENTION_REGEX.sub('', text)
    text = HASHTAG_REGEX.sub('', text)
    text = SPECIAL_CHAR_REGEX.sub('', text) 
    text = text.lower()
    words = word_tokenize(text)
    words = [LEMMATIZER.lemmatize(word) for word in words if word not in STOP_WORDS]
    text = ' '.join(words)
    text = text.strip()
    return text

In [162]:
english_tweets['text']

0         #UPDATE The latest on the war in #Ukraine -- h...
1         Russia calls on EU, NATO to stop arms supplies...
2         A windy freeze sends a shiver. The ground shak...
3         #Ukraine: Footage of Ukrainian forces hitting ...
4         JUST IN - VISA suspends all operations in #Rus...
                                ...                        
362800    This made a difference. The It factor . Did th...
362801    This made a difference. The It factor . Did th...
362802    @errollouis ❤️ Pushkin was discussed in this c...
362803    Can anyone take the tracks apart? #StopPutinNO...
362804    This made a difference. The It factor . Did th...
Name: text, Length: 325640, dtype: object

In [169]:
english_tweets.loc[:, 'text'] = english_tweets['text'].apply(clean_text)

/tmp/ipykernel_1008/2363082672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_tweets.loc[:, 'text'] = english_tweets['text'].apply(clean_text)


In [170]:
english_tweets['text']

0         latest war ukraine asks west military assistan...
1         russia call eu nato stop arm supply ukraine st...
2         windy freeze sends shiver ground shake artille...
3         footage ukrainian force hitting target possibl...
4         visa suspends operation card issued russia lon...
                                ...                        
362800                     made difference factor beginning
362801                     made difference factor beginning
362802    pushkin discussed climate never suffered short...
362803                              anyone take track apart
362804                     made difference factor beginning
Name: text, Length: 325640, dtype: object

In [175]:
english_tweets.to_csv('final_dataset', index = False)